In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


In [5]:
df = pd.read_csv('data/df4.csv')
rating = df.copy()
rating.drop(['customer_id','article_id', 'count'], axis=1)
rating = rating[['user_id', 'item_id', 'rfm']]
rating = rating.rename(columns={'rfm':'rating'})
rating

,user_id,item_id,rating
0,2,40179,5.472143
1,2,10520,3.484524
2,7,6387,1.993809
3,7,46304,1.993809
4,7,46305,1.993809
...,...,...,...
31788319,1371691,104961,9.503095
31788320,1371691,100629,10.000000
31788321,1371721,104053,9.503095
31788322,1371747,88521,5.527857


In [6]:
rating.to_csv('data/sgd.csv', index=False)

In [2]:
class MovielensData:
    def __init__(self, data_file_path, train_percent=0.8):
        self.data_file_path = data_file_path
        df = pd.read_csv('data/sgd.csv')

        self.n_users = df.user_id.max()
        self.n_items = df.item_id.max()
        print ('Number of users = ' + str(self.n_users) + ' | Number of movies = ' + str(self.n_items))

        self.R = np.zeros((self.n_users+1, self.n_items+1))
        self.T = np.zeros((self.n_users+1, self.n_items+1))
        print("zeros")
        self.data = pd.DataFrame(df)
        random.seed(0)
        for line in tqdm(self.data.itertuples()):
            if (random.random() < train_percent):
                self.R[line[1], line[2]] = line[3]
            else:
                self.T[line[1], line[2]] = line[3] 


In [ ]:
import numpy as np

In [3]:
# Calculate the RMSE
def rmse(I,R,U,M):
    return np.sqrt(np.sum((I * (R - np.dot(U,M.T)))**2)/len(R[R > 0]))

class SdgMF():
    def __init__(self, R, T, rank=20, reg=0.1, n_iter=10, learning_rate=0.001):
        self.R = R
        self.T = T
        self.n_users, self.n_items = R.shape
        self.rank = rank
        self.reg = reg
        self.n_iter = n_iter
        self.learning_rate = learning_rate

        self.sample_row, self.sample_col = self.R.nonzero()
        self.n_samples = len(self.sample_row)

        # Index matrix for training data
        I = self.R.copy()
        I[I > 0] = 1
        I[I == 0] = 0

        self.I = I

        # Index matrix for test data
        I2 = self.T.copy()
        I2[I2 > 0] = 1
        I2[I2 == 0] = 0

        self.I2 = I2

    def train(self):
        self.user_vecs = np.random.rand(self.n_users, self.rank)
        self.item_vecs = np.random.rand(self.n_items, self.rank)
        
        np.random.seed(0)
        for iter in tqdm(range(self.n_iter)):
            self.sdg()

            train_rmse = rmse(self.I, self.R, self.user_vecs, self.item_vecs)
            test_rmse = rmse(self.I2, self.T, self.user_vecs, self.item_vecs)
            print("tran_rsme:", train_rmse, "test_rmse:", test_rmse)
    
    def sdg(self):
        training_indices = np.arange(self.n_samples)
        np.random.shuffle(training_indices)
        for idx in tqdm(training_indices):
            u, i = self.sample_row[idx], self.sample_col[idx]
            e = self.R[u, i] - self.user_vecs[u,:].dot(self.item_vecs[i,:].T)

            self.user_vecs[u, :] += self.learning_rate * \
                    (e*self.item_vecs[i,:]- self.reg*self.user_vecs[u,:]) 
            self.item_vecs[i, :] += self.learning_rate * \
                    (e*self.user_vecs[u,:]- self.reg*self.item_vecs[i,:])


if __name__ == "__main__":
    md = MovielensData('data/sgd.csv')
    mf = SdgMF(md.R, md.T, n_iter=20, learning_rate=0.001)
    mf.train()

Number of users = 1371979 | Number of movies = 105539
zeros


11624234it [00:44, 238162.75it/s]

: 

: 